In [89]:
import torch

from torch import nn
from torch.autograd import Variable

In [90]:
input_parameters = 5
layer_size = 5
layer_count = 3
classification_size = 1
learning_rate = 0.001

In [91]:
graph_stat = Variable(torch.ones(input_parameters, layer_size))

In [92]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.linear0 = torch.nn.Linear(layer_size, layer_size)
        self.linear1 = torch.nn.Linear(layer_size, layer_size)
        self.linear2 = torch.nn.Linear(layer_size, classification_size)

    def forward(self, x):
        x = self.linear0(x)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

In [93]:
model = NeuralNet()

result = model(graph_stat)
result

tensor([[-0.0821],
        [-0.0821],
        [-0.0821],
        [-0.0821],
        [-0.0821]], grad_fn=<AddmmBackward0>)

In [94]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [114]:
samples = 1000

def f(i, j) -> list[float]:
    return i * 10 + j * j

dataset = [torch.Tensor([f(i, j) for j in range(input_parameters)]) + torch.randn(input_parameters) for i in range(samples)]
dataset_labels = [torch.Tensor([i % 3]) for i in range(samples)]

In [115]:
bound = int(samples * 0.8)
train = list(zip(dataset[:bound], dataset_labels[:bound]))
test = list(zip(dataset[bound:], dataset_labels[bound:]))

In [117]:
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
total_step = len(train)
for epoch in range(num_epochs):
    for i, (image, label) in enumerate(train):
        # Move tensors to the configured device
        image = image.to(device)
        label = label.to(device)

        # Forward pass
        outputs = model(image)
        loss = criterion(outputs, label)

        # Backpropagation and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/10], Step [100/800], Loss: -0.0000
Epoch [1/10], Step [200/800], Loss: -0.0000
Epoch [1/10], Step [300/800], Loss: -0.0000
Epoch [1/10], Step [400/800], Loss: -0.0000
Epoch [1/10], Step [500/800], Loss: -0.0000
Epoch [1/10], Step [600/800], Loss: -0.0000
Epoch [1/10], Step [700/800], Loss: -0.0000
Epoch [1/10], Step [800/800], Loss: -0.0000
Epoch [2/10], Step [100/800], Loss: -0.0000
Epoch [2/10], Step [200/800], Loss: -0.0000
Epoch [2/10], Step [300/800], Loss: -0.0000
Epoch [2/10], Step [400/800], Loss: -0.0000
Epoch [2/10], Step [500/800], Loss: -0.0000
Epoch [2/10], Step [600/800], Loss: -0.0000
Epoch [2/10], Step [700/800], Loss: -0.0000
Epoch [2/10], Step [800/800], Loss: -0.0000
Epoch [3/10], Step [100/800], Loss: -0.0000
Epoch [3/10], Step [200/800], Loss: -0.0000
Epoch [3/10], Step [300/800], Loss: -0.0000
Epoch [3/10], Step [400/800], Loss: -0.0000
Epoch [3/10], Step [500/800], Loss: -0.0000
Epoch [3/10], Step [600/800], Loss: -0.0000
Epoch [3/10], Step [700/800], Lo

In [118]:
# Test the model
# In the test phase, don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for image, label in test:
        image = image.to(device)
        label = label.to(device)
        outputs = model(image)
        _, predicted = torch.max(outputs.data, 0)
        total += label.size(0)
        correct += (predicted == label).item()

    print(f'Accuracy of the network on the {len(test)} test images: {100 * correct / total} %')

Accuracy of the network on the 200 test images: 33.5 %
